<a href="https://colab.research.google.com/github/JSJeong-me/GPT-DB/blob/main/01-musinsa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!cp /content/drive/MyDrive/inner-people/musinsa-review.csv .

In [2]:
!pip install langchain
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install openai
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.1/212.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 12.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [4]:
!echo "OPENAI_API_KEY=sk-" >> .env
!source /content/.env

In [5]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
# Access the API key using the variable name defined in the .env file
api_key = os.getenv("OPENAI_API_KEY")

In [6]:
import csv
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [7]:
# Define the columns we want to embed vs which ones we want in metadata
columns_to_embed = ["Description","Features"]
columns_to_metadata = ["unique_id","product", "review"]

In [8]:
# Process the CSV into the embedable content vs the metadata and put it into Document format so that we can chunk it into pieces.
docs = []
with open('musinsa-review.csv', newline="", encoding='utf-8') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    for i, row in enumerate(csv_reader):
        to_metadata = {col: row[col] for col in columns_to_metadata if col in row}
        values_to_embed = {k: row[k] for k in columns_to_embed if k in row}
        to_embed = "\n".join(f"{k.strip()}: {v.strip()}" for k, v in values_to_embed.items())
        newDoc = Document(page_content=to_embed, metadata=to_metadata)
        docs.append(newDoc)

In [11]:
len(docs)

409834

In [12]:
# Lets split the document using Chracter splitting.
splitter = CharacterTextSplitter(separator = "\n",
                                chunk_size=500,
                                chunk_overlap=0,
                                length_function=len)
documents = splitter.split_documents(docs)

In [13]:
# import
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma

In [ ]:
# # create the open-source embedding function
# embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# # load it into Chroma
# db = Chroma.from_documents(docs, embedding_function)

In [14]:
!pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [ ]:
from langchain_openai import OpenAI
# # Generate embeddings from documents and store in a vector database
embeddings_model = OpenAIEmbeddings()
db = Chroma.from_documents(documents, OpenAIEmbeddings())

In [ ]:
!pip install sentence-transformers

In [ ]:
# import csv
# from langchain.docstore.document import Document
# from langchain.text_splitter import CharacterTextSplitter
# from langchain.embeddings import OpenAIEmbeddings
# from langchain.vectorstores import Chroma

In [ ]:
# Define the columns we want to embed vs which ones we want in metadata
# columns_to_embed = ["Description","Features"]
# columns_to_metadata = ["Product Name","Price", "Rating","Description", "Features"]

In [ ]:
# Process the CSV into the embedable content vs the metadata and put it into Document format so that we can chunk it into pieces.
# docs = []
# with open('TestListings.csv', newline="", encoding='utf-8-sig') as csvfile:
#     csv_reader = csv.DictReader(csvfile)
#     for i, row in enumerate(csv_reader):
#         to_metadata = {col: row[col] for col in columns_to_metadata if col in row}
#         values_to_embed = {k: row[k] for k in columns_to_embed if k in row}
#         to_embed = "\n".join(f"{k.strip()}: {v.strip()}" for k, v in values_to_embed.items())
#         newDoc = Document(page_content=to_embed, metadata=to_metadata)
#         docs.append(newDoc)

In [ ]:
# Lets split the document using Chracter splitting.
# splitter = CharacterTextSplitter(separator = "\n",
#                                 chunk_size=500,
#                                 chunk_overlap=0,
#                                 length_function=len)
# documents = splitter.split_documents(docs)

In [ ]:
# Generate embeddings from documents and store in a vector database
# embeddings_model = OpenAIEmbeddings()
# db = Chroma.from_documents(documents, OpenAIEmbeddings())

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [ ]:
# Query the vector database for information.
query = "Heart rate monitor"
docs = db.similarity_search(query)
print(docs[0].page_content)
print(docs[0].metadata)

Description: Achieve your fitness goals with our Fitness Tracker Smartwatch. Monitor your activity, heart rate, and receive notifications on your wrist.
Features: - Heart rate monitor and activity tracking.<br> - Built-in GPS for accurate workout tracking.<br> - Sleep analysis and guided breathing exercises.<br> - Receive notifications from your smartphone.<br> - Water-resistant for workouts and everyday use.<br> - Long battery life.
{'Description': 'Achieve your fitness goals with our Fitness Tracker Smartwatch. Monitor your activity, heart rate, and receive notifications on your wrist.', 'Features': '- Heart rate monitor and activity tracking.<br> - Built-in GPS for accurate workout tracking.<br> - Sleep analysis and guided breathing exercises.<br> - Receive notifications from your smartphone.<br> - Water-resistant for workouts and everyday use.<br> - Long battery life.', 'Price': '149.99', 'Product Name': 'Fitness Tracker Smartwatch', 'Rating': '4.5'}


In [ ]:
query = "what is the product name of the Min rating?"
docs = db.similarity_search(query)
print(docs[0].page_content)
print(docs[0].metadata)

Features: - High-fidelity sound with deep bass and clear treble. - Noise-canceling technology for an immersive listening experience. - 20 hours of playtime on a single charge. - Ergonomic design for a secure and comfortable fit. - Touch controls for easy playback and calls. - IPX7 waterproof rating for workouts and outdoor activities. - Built-in microphone for hands-free calls.
{'Description': "Elevate your music experience with our Ultimate Wireless Bluetooth Earbuds. These earbuds are designed for audiophiles who demand premium sound quality and convenience. With advanced Bluetooth 5.0 technology, you'll enjoy seamless connectivity and crystal-clear audio.", 'Features': '- High-fidelity sound with deep bass and clear treble. - Noise-canceling technology for an immersive listening experience. - 20 hours of playtime on a single charge. - Ergonomic design for a secure and comfortable fit. - Touch controls for easy playback and calls. - IPX7 waterproof rating for workouts and outdoor act

Advanced Querying

In [ ]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

# Metadata schema based on the values on the CSV
metadata_field_info = [
    AttributeInfo(
        name="Product Name",
        description="Name of the product",
        type="string",
    ),
    AttributeInfo(
        name="Price",
        description="The price of the product as a number. Ex. 149.99",
        type="string",
    ),
    AttributeInfo(
        name="Rating",
        description="The rating of the product as a number from 0 to 5. Ex. 4.5",
        type="string",
    ),
    AttributeInfo(
        name="Description",
        description="Description of the product", type="string"
    ),
    AttributeInfo(
        name="Features",
        description="Features of the product",
        type="string"
    ),
]
document_content_description = "Product listing"

In [ ]:
# Configure retriver
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(llm, db, document_content_description, metadata_field_info, verbose=True)

In [ ]:
# Retrieve values
retriever.get_relevant_documents("good heart monitor")

# The result is:
# Description: Achieve your fitness goals with our Fitness Tracker Smartwatch. Monitor your activity, heart rate, and receive notifications on your wrist.\n
# Features: - Heart rate monitor and activity tracking.<br> - Built-in GPS for accurate workout tracking.<br> - Sleep analysis and guided breathing exercises.<br> - Receive notifications from your smartphone.<br> - Water-resistant for workouts and everyday use.<br> - Long battery life.',
# metadata={
#    'Product Name': 'Fitness Tracker Smartwatch',
#    'Price': '149.99',
#    'Rating': '4.5',
#    'Description': 'Achieve your fitness goals with our Fitness Tracker Smartwatch. Monitor your activity, heart rate, and receive notifications on your wrist.',
#    'Features': '- Heart rate monitor and activity tracking.<br> - Built-in GPS for accurate workout tracking.<br> - Sleep analysis and guided breathing exercises.<br> - Receive notifications from your smartphone.<br> - Water-resistant for workouts and everyday use.<br> - Long battery life.'
# }